# EM Circle Denoiser: Simple Example

This notebook demonstrates generating noisy EM-style circles and denoising them with a CNN autoencoder.

In [ ]:
# Imports
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from skimage.measure import find_contours
from dataset import CircleDatasetEM
from model import Denoiser
from utils import weighted_mse

## 1. Prepare dataset

In [ ]:
dataset = CircleDatasetEM(num_samples=10)
loader = DataLoader(dataset, batch_size=2, shuffle=True)

## 2. Initialize model and optimizer

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Denoiser().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

## 3. Simple training loop (just a few iterations for demo)

In [ ]:
for epoch in range(2):  # just 2 epochs for demonstration
    for noisy, clean in loader:
        noisy, clean = noisy.to(device), clean.to(device)
        out = model(noisy)
        loss = weighted_mse(out, clean, weight=10.0)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1} done, loss={loss.item():.4f}')

## 4. Visualize some examples

In [ ]:
for i in range(3):
    noisy, clean = dataset[i]
    noisy_batch = noisy.unsqueeze(0).to(device)
    with torch.no_grad():
        denoised = model(noisy_batch).cpu().squeeze().numpy()
    
    contours = find_contours(clean.squeeze().numpy(), 0.5)
    fig, axes = plt.subplots(1, 4, figsize=(16,4))
    titles = ['Noisy 1', 'Noisy 2', 'Noisy 3', 'Denoised']
    imgs = [noisy[0].numpy(), noisy[1].numpy(), noisy[2].numpy(), denoised]
    
    for ax, img, title in zip(axes, imgs, titles):
        ax.imshow(img, cmap='gray')
        ax.set_title(title)
        ax.axis('off')
        for contour in contours:
            ax.plot(contour[:,1], contour[:,0], color='red', linewidth=2)
    plt.show()

### ✅ Notes
- This notebook trains the model for **very few iterations** just for demonstration.
- Red contours highlight the original circle.
- 3 noisy versions of the same circle are denoised into a single clean output.